In [1]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset
from openai import OpenAI

# Configuration
max_seq_length = 2048 # Adjust based on your review lengths
dtype = None
load_in_4bit = True # Use 4-bit quantization to save memory

# Load the Typhoon Model
# using a common alias or the likely path: 'scb10x/typhoon-v2-7b-instruct' or similar
model_name = "scb10x/typhoon-v1.5-8b-instruct"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Load and Format Your Data
dataset = load_dataset("json", data_files="training_data.jsonl", split="train")

def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return { "text" : texts }

dataset = dataset.map(formatting_prompts_func, batched = True,)

# Add LoRA Adapters (Fine-tuning configuration)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

# Initialize Trainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can set to True for speed boost
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Adjust based on dataset size
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

# Train the model
trainer_stats = trainer.train()

# Save the model
model.save_pretrained("typhoon_fake_review_detector")
tokenizer.save_pretrained("typhoon_fake_review_detector")

print("Fine-tuning complete. Adapters saved to 'typhoon_fake_review_detector'.")

NotImplementedError: Unsloth currently only works on NVIDIA, AMD and Intel GPUs.

In [ ]:
from unsloth import FastLanguageModel
import torch

# Configuration
# Do when app start running??
max_seq_length = 2048
dtype = None
load_in_4bit = True

# Load the Fine-Tuned Model
model_path = "typhoon_fake_review_detector"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_path,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# -------------------------------------------

In [ ]:


FastLanguageModel.for_inference(model)

# should be the same System Prompt format as our training data
system_prompt = "You are Typhoon, a model that analyzes product reviews and outputs the likelihood of being fake."
test_review = "kuay"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": f"""
    PRODUCT_TITLE: \"Targus PAUK10U Ultra Mini USB Keypad, Black\",
    REVIEW_TITLE: \"useful\" ,
    Review: \"{test_review}\"
    """}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must be True for generation
    return_tensors = "pt",
).to("cuda")

# Generate Response
outputs = model.generate(
    input_ids = inputs,
    max_new_tokens = 64, # Adjust-able
    use_cache = True,
    temperature = 0.1,
)

# Decode and Print
# ทำยังไงให้มันไม่ print สะเปะสะปะ วะ
decoded_output = tokenizer.batch_decode(outputs)
print("\n--- Model Prediction ---")
print(decoded_output[0])


--- Model Prediction ---
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are Typhoon, a model that analyzes product reviews and outputs the likelihood of being fake.<|eot_id|><|start_header_id|>user<|end_header_id|>

PRODUCT_TITLE: "Targus PAUK10U Ultra Mini USB Keypad, Black",
    REVIEW_TITLE: "useful",
    Review: "kuay"<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{"fake_probability": 0.8}<|eot_id|>
